In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey
from haversine import haversine

In [2]:
# load standard prepared data fail
fire_data = pd.read_csv("Fire.csv")
fire_data.head()

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng,TOTAL LOSS
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,-5000.0,0.0,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783,5000
1,1,17962.0,10/30/2019,03:42 AM,75229.0,7,A,13.0,DWELLING,1.0,...,-10000.0,0.0,525470.0,1.87,NaN,NaN,10728 LATHROP DR,32.894248,-96.857108,10000
2,2,17963.0,10/30/2019,10:47 AM,75241.0,5,B,3.0,DWELLING,1.0,...,-5000.0,-15000.0,286000.0,6.54,NaN,NaN,6205 S R L THORNTON SERV,32.694543,-96.824020,20000
3,3,17964.0,10/30/2019,05:32 PM,75243.0,2,B,10.0,DWELLING,1.0,...,-8000.0,-4000.0,114710.0,9.47,NaN,NaN,12044 WILLOWDELL DR,32.913584,-96.764435,12000
4,4,17965.0,10/30/2019,11:03 AM,75212.0,9,B,6.0,DWELLING,1.0,...,-45000.0,-10000.0,29000.0,65.48,NaN,NaN,2212 LAPSLEY ST,32.769986,-96.908816,55000


In [3]:
#Search for 2 nearest fire stations, find latitude and longitude
places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# needed to clear output below
from IPython.display import clear_output

dist0 = []
dist1 = []
length = len(fire_data)
status = ""

for index, row in fire_data.iterrows():
    
    # store current lat long
    lat = row['Lat']
    lng = row['Lng']
    latlng = str(lat) + ',' + str(lng)
    
    params = {
        "location": latlng,
        "key": gkey,
        "type": "fire_station",
        'rankby': 'distance'
    }
    
    try:
        geo_data = requests.get(places_url, params=params).json()
        
        # record lat & long for first two results
        lat0 = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng0 = geo_data["results"][0]["geometry"]["location"]["lng"]
        lat1 = geo_data["results"][1]["geometry"]["location"]["lat"]
        lng1 = geo_data["results"][1]["geometry"]["location"]["lng"]
        if (geo_data['status'] == 'INVALID_REQUEST'):
            status = "Invalid Request."
        else:
            status = 'found.'
        
        dist0.append(haversine(lat,lng,lat0,lng0,'mi'))
        dist1.append(haversine(lat,lng,lat1,lng1,'mi'))
        
    except:
        lat0 = None
        lng0 = None
        lat1 = None
        lng1 = None
        status = 'error.'
        dist0.append(None)
        dist1.append(None)
    
    # Show status while running, but without the clutter.
    clear_output(wait=True)
    print(f"{index+1} of {length}: {status}  Dist0: {dist0[index]}, Dist1: {dist1[index]}")
    
    
    

621 of 621: found.  Dist0: 1.1872932191257024, Dist1: 1.7557976327439737


In [ ]:
dist0

In [ ]:
dist1

In [ ]:
geo_data